### Load Libraries

In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV


from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, PolynomialFeatures, StandardScaler
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor

from sklearn.linear_model import LinearRegression, PoissonRegressor, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_log_error, make_scorer

sns.set_theme(style='darkgrid',font_scale=1.3)

### Load Data

In [ ]:
df = pd.read_csv('../data/train.csv',index_col=0, parse_dates=True)
df.head()

In [ ]:
df.info()

In [ ]:
# check missing values
df.isna().sum()

### Data Preprocessing

#### Extract time features

In [ ]:
df['year'] = df.index.year
df['month'] = df.index.month
df['day'] = df.index.day
df['weekday'] = df.index.weekday
df['hour'] = df.index.hour
df.head()

### Exploratory Data Analysis

#### Correlation Analysis of Features

In [ ]:
df.head()
plt.figure(figsize=(18,10))
corr = df.drop(['casual','registered'],axis=1).corr()
sns.heatmap(corr, annot=True)

There is high correlation between ```month``` and ```season```, ```weekday``` and ```workingday```, ```temp``` and ```atemp```

Therefore we can drop ``` workingday```, ```season``` and ```atemp```  

In [ ]:
# Drop 'workingday', 'season', and 'atemp' 
df.drop(['workingday', 'season', 'atemp'], axis = 1, inplace = True)

#### Analyze Average of Counts Per Time 

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(8, 20))
ax = ax.flatten() 
features = ['month','day','hour']

for index,feature in enumerate(features):
        
        sns.lineplot(ax=ax[index],x=feature,y='casual',data=df.groupby(feature)['casual'].mean().reset_index(),label ='casual')
        sns.lineplot(ax=ax[index],x=feature,y='registered',data=df.groupby(feature)['registered'].mean().reset_index(),label= 'registered')
        sns.lineplot(ax=ax[index],x=feature,y='count',data=df.groupby(feature)['count'].mean().reset_index(),label = 'count')
        ax[index].legend(loc='upper left')
        ax[index].set_ylabel('')
        ax[index].set_title(f'Average of Counts Per {feature}')

#### Analyze Hourly Trends

In [ ]:
# Hourly trends of count of bikes during weekdays and holdays
fig = plt.figure(figsize=(8,6))
ax = sns.pointplot(data=df, x='hour', y='count', hue='holiday');
handles, labels  =  ax.get_legend_handles_labels()
ax.legend(handles,['workingday','holiday']);

In [ ]:
# Hourly trends of count of bikes with differet weather condition (1 = good -> 4 = very bad)
fig = plt.figure(figsize=(8,6))
sns.pointplot(data=df, x="hour", y="count", hue="weather");

#### Analyze Affect of Selected Features on Count

##### Environmental Conditions

In [ ]:
# Counts depending on weather
plt.figure(figsize=(15,8))
ax = sns.violinplot(x=df['weather'].astype(str),y=df['count'], size = 4)
plt.show()

In [ ]:
# Counts depending on temp, humidity and, windspeed
sns.pairplot(data = df,
             x_vars = ['temp', 'humidity', 'windspeed'],
             y_vars = ['casual', 'registered', 'count'],
             kind='reg')

People prefer more cycling as the days get hotter and prefere less cycling as the days get wetter

##### Other Features

In [ ]:
# Counts depending on holiday (holiday = 1)
plt.figure(figsize=(8,6))
sns.boxplot(data = df, x='holiday', y='count')

### Linear Regression

In [ ]:
# define features and target
X = df.drop(['count','registered','casual'], axis=1)
y = df['count']

In [ ]:
# Train-test split of the data
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=100)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

#### Feature Engineering

Numeric columns should be scaled, categorical columns should be onehot encoded, use of polynomial features

##### Define Pipelines

In [ ]:
# Scaling for numeric variables
num_pipe = make_pipeline(
    MinMaxScaler()
)

In [ ]:
# One_hot_encoding for categorical variables
cat_pipe = make_pipeline(
    OneHotEncoder(handle_unknown = 'ignore')
)

In [ ]:
# Define preprocessor
feature_transform = ColumnTransformer([
    ('do_nothing', 'passthrough', ['holiday']), 
    ('one_hot_encoding', cat_pipe, ['weather','year', 'month', 'day', 'hour']),
    ('scaling', num_pipe, ['temp','humidity', 'windspeed'])
])

#### Create Model

In [ ]:
lr = Pipeline([
            ('preprocessor', feature_transform),
            ('poly_features', PolynomialFeatures(degree=2, 
                                                 include_bias=False, 
                                                 interaction_only=False)),
            ('m_lr', LinearRegression())
])

# Do target transformation
lr_t = TransformedTargetRegressor(regressor=lr, func=np.log1p, inverse_func=np.expm1)

In [57]:
lr_t.fit(X_train, y_train)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=Pipeline(steps=[('preprocessor',
                                                      ColumnTransformer(transformers=[('do_nothing',
                                                                                       'passthrough',
                                                                                       ['holiday']),
                                                                                      ('one_hot_encoding',
                                                                                       Pipeline(steps=[('onehotencoder',
                                                                                                        OneHotEncoder(handle_unknown='ignore'))]),
                                                                                       ['weather',
                                                                                        'year',
                                                                                        'month',
                                                                                        'day',
                                                                                        'hour']),
                                                                                      ('scaling',
                                                                                       Pipeline(steps=[('minmaxscaler',
                                                                                                        MinMaxScaler())]),
                                                                                       ['temp',
                                                                                        'humidity',
                                                                                        'windspeed'])])),
                                                     ('poly_features',
                                                      PolynomialFeatures(include_bias=False)),
                                                     ('m_lr',
                                                      LinearRegression())]))

#### Evaluate Model

In [58]:
# Define rmsle function for evaluation
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

In [59]:
# Transform rmsle fucntion into scorer 
rmsle_scorer = make_scorer(score_func=rmsle, greater_is_better=False)

In [60]:
def print_evaluation_metrics(model,X_train,y_train,X_test,y_test):
    """ Print out R2 and RMSLE as evaluation metrics """
    
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    print(f'R2 train : {round(model.score(X_train,y_train),2)}')
    print(f'R2 test: {round(model.score(X_test,y_test),2)}')
    print('\n')
    print(f'RMSLE train: {round(rmsle(y_train, y_pred_train),2)}')
    print(f'RMSLE test: {round(rmsle(y_test, y_pred_test),2)}')

In [61]:
print_evaluation_metrics(lr_t,X_train,y_train,X_test,y_test)

R2 train : 0.79
R2 test: 0.72


RMSLE train: 0.52
RMSLE test: 0.63


In [64]:
# # save evaluation metrics using cross validation
# def save_evaluation_metrics(model,model_name, summary, X, y):
    
#     R2 = round(cross_val_score(model, X, y, cv = 4, scoring='r2').mean(),2)
#     RMSLE = round(cross_val_score(model, X, y, cv = 4, scoring=rmsle_scorer).mean(),2)
    
    
#     summary['Model Name'].append(model_name)
#     summary['R2'].append(R2)
    
    
#     return summary

In [66]:
# summary = {'Model Name':[],'R2':[],'RMSLE':[]}

# summary = save_evaluation_metrics(lr_t,'Linear Regressin',summary,X,y)
# summary

### Ridge Regression

#### Feature Engineering

Same as linear regression

#### Create Model

In [116]:
rr = Pipeline([
    ('preprocessor', feature_transform),
    ('poly_features', PolynomialFeatures(degree=2, 
                                         include_bias=False,
                                         interaction_only=False)),
    ('m_rr', Ridge(alpha=1))
])
# Do target transformation
rr_t = TransformedTargetRegressor(regressor=rr, func=np.log1p, inverse_func=np.expm1)

In [102]:
rr_t.fit(X_train,y_train)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=Pipeline(steps=[('preprocessor',
                                                      ColumnTransformer(transformers=[('do_nothing',
                                                                                       'passthrough',
                                                                                       ['holiday']),
                                                                                      ('one_hot_encoding',
                                                                                       Pipeline(steps=[('onehotencoder',
                                                                                                        OneHotEncoder(handle_unknown='ignore'))]),
                                                                                       ['weather',
                                                                                        'year',
                                                                                        'month',
                                                                                        'day',
                                                                                        'hour']),
                                                                                      ('scaling',
                                                                                       Pipeline(steps=[('minmaxscaler',
                                                                                                        MinMaxScaler())]),
                                                                                       ['temp',
                                                                                        'humidity',
                                                                                        'windspeed'])])),
                                                     ('poly_features',
                                                      PolynomialFeatures(include_bias=False)),
                                                     ('m_rr', Ridge(alpha=1))]))

#### Evaluate Model

In [103]:
print_evaluation_metrics(rr_t,X_train,y_train,X_test,y_test)

R2 train : 0.79
R2 test: 0.72


RMSLE train: 0.52
RMSLE test: 0.62


#### Tune Hyperparameters

In [129]:
# Function to print out results of the hyperparameter optimization
def print_results(cv_h, show_all = False):
    print('Best:')
    print(round(cv_h.best_score_, 3), ' for ', cv_h.best_params_)
    if show_all:
        print('All:')
        for mean, params in zip(cv_h.cv_results_['mean_test_score'], 
                                cv_h.cv_results_['params']):
                print('{}  for {}'.format(round(mean, 3), params))

In [111]:
# get list of hyperparameters
rr_t.get_params().keys()

dict_keys(['check_inverse', 'func', 'inverse_func', 'regressor__memory', 'regressor__steps', 'regressor__verbose', 'regressor__preprocessor', 'regressor__poly_features', 'regressor__m_rr', 'regressor__preprocessor__n_jobs', 'regressor__preprocessor__remainder', 'regressor__preprocessor__sparse_threshold', 'regressor__preprocessor__transformer_weights', 'regressor__preprocessor__transformers', 'regressor__preprocessor__verbose', 'regressor__preprocessor__verbose_feature_names_out', 'regressor__preprocessor__do_nothing', 'regressor__preprocessor__one_hot_encoding', 'regressor__preprocessor__scaling', 'regressor__preprocessor__one_hot_encoding__memory', 'regressor__preprocessor__one_hot_encoding__steps', 'regressor__preprocessor__one_hot_encoding__verbose', 'regressor__preprocessor__one_hot_encoding__onehotencoder', 'regressor__preprocessor__one_hot_encoding__onehotencoder__categories', 'regressor__preprocessor__one_hot_encoding__onehotencoder__drop', 'regressor__preprocessor__one_hot_enc

In [151]:
parameters = {'regressor__m_rr__alpha': np.arange(1e-2,50,0.5)}

In [161]:
cv_h =  GridSearchCV(estimator=rr_t,param_grid=parameters,cv=5,n_jobs=-1)

In [162]:
cv_h.fit(X_train,y_train)

GridSearchCV(cv=5,
             estimator=TransformedTargetRegressor(func=<ufunc 'log1p'>,
                                                  inverse_func=<ufunc 'expm1'>,
                                                  regressor=Pipeline(steps=[('preprocessor',
                                                                             ColumnTransformer(transformers=[('do_nothing',
                                                                                                              'passthrough',
                                                                                                              ['holiday']),
                                                                                                             ('one_hot_encoding',
                                                                                                              Pipeline(steps=[('onehotencoder',
                                                                                                                               OneHotEncoder(handle_unknown='ignore'))]),
                                                                                                              ['weather',
                                                                                                               'year',
                                                                                                               'month',
                                                                                                               'day',
                                                                                                               'hour...
       3.001e+01, 3.051e+01, 3.101e+01, 3.151e+01, 3.201e+01, 3.251e+01,
       3.301e+01, 3.351e+01, 3.401e+01, 3.451e+01, 3.501e+01, 3.551e+01,
       3.601e+01, 3.651e+01, 3.701e+01, 3.751e+01, 3.801e+01, 3.851e+01,
       3.901e+01, 3.951e+01, 4.001e+01, 4.051e+01, 4.101e+01, 4.151e+01,
       4.201e+01, 4.251e+01, 4.301e+01, 4.351e+01, 4.401e+01, 4.451e+01,
       4.501e+01, 4.551e+01, 4.601e+01, 4.651e+01, 4.701e+01, 4.751e+01,
       4.801e+01, 4.851e+01, 4.901e+01, 4.951e+01])})

In [163]:
# Print results of hyperparameter optimization using random search       
print_results(cv_h, show_all=False)

Best:
0.739  for  {'regressor__m_rr__alpha': 34.01}


### Lasso Regression

#### Featrure Engineering

same as linear regression

#### Create Model

In [187]:
lar = Pipeline([
    ('preprocessor', feature_transform),
    #('poly_features', PolynomialFeatures(degree=1, 
    #                                     include_bias=False,
    #                                     interaction_only=False)),
    ('m_lar', Lasso(alpha=0.1))
])
# Do target transformation
lar_t = TransformedTargetRegressor(regressor=lar, func=np.log1p, inverse_func=np.expm1)

In [188]:
lar_t.fit(X_train,y_train)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=Pipeline(steps=[('preprocessor',
                                                      ColumnTransformer(transformers=[('do_nothing',
                                                                                       'passthrough',
                                                                                       ['holiday']),
                                                                                      ('one_hot_encoding',
                                                                                       Pipeline(steps=[('onehotencoder',
                                                                                                        OneHotEncoder(handle_unknown='ignore'))]),
                                                                                       ['weather',
                                                                                        'year',
                                                                                        'month',
                                                                                        'day',
                                                                                        'hour']),
                                                                                      ('scaling',
                                                                                       Pipeline(steps=[('minmaxscaler',
                                                                                                        MinMaxScaler())]),
                                                                                       ['temp',
                                                                                        'humidity',
                                                                                        'windspeed'])])),
                                                     ('m_lar',
                                                      Lasso(alpha=0.1))]))

#### Evaluate Model

In [189]:
print_evaluation_metrics(lar_t,X_train,y_train,X_test,y_test)

R2 train : -0.23
R2 test: -0.2


RMSLE train: 1.37
RMSLE test: 1.4


### Poisson Regression

#### Feature Engineering

Same as linear regression

#### Create Model

In [69]:
pr = Pipeline([
    ('preprocessor', feature_transform),
    ('poly_features', PolynomialFeatures(degree=2, 
                                         include_bias=False,
                                         interaction_only=False)),
    ('m_pr', PoissonRegressor(alpha=0))
])

In [70]:
pr.fit(X_train, y_train)

/Users/milad/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_glm/glm.py:294: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('do_nothing', 'passthrough',
                                                  ['holiday']),
                                                 ('one_hot_encoding',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['weather', 'year', 'month',
                                                   'day', 'hour']),
                                                 ('scaling',
                                                  Pipeline(steps=[('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['temp', 'humidity',
                                                   'windspeed'])])),
                ('poly_features', PolynomialFeatures(include_bias=False)),
                ('m_pr', PoissonRegressor(alpha=0))])

#### Evaluate Model

In [71]:
print_evaluation_metrics(pr,X_train,y_train,X_test,y_test)

R2 train : 0.86
R2 test: 0.79


RMSLE train: 0.57
RMSLE test: 0.64


### Random Forest Regression

#### Feature Engineering

Not required

#### Create Model

In [74]:
rfr = TransformedTargetRegressor(regressor=RandomForestRegressor(),func=np.log1p, inverse_func=np.expm1)

In [75]:
rfr.fit(X_train,y_train)

TransformedTargetRegressor(func=<ufunc 'log1p'>, inverse_func=<ufunc 'expm1'>,
                           regressor=RandomForestRegressor())

#### Evaluate Model

In [77]:
print_evaluation_metrics(rfr,X_train,y_train,X_test,y_test)

R2 train : 0.99
R2 test: 0.94


RMSLE train: 0.11
RMSLE test: 0.32


In [ ]:
# Feature Importance
